In [1]:
import numpy as np

In [2]:
#sign: given x, if x<0 return -1, else return 1
def sign(x):
    if x<0:
        return -1
    else:
        return 1

In [3]:
def genNoise(noise_rate, data_size, x):
    #在[-0.2,0.8]產生隨機變數，再取sign => 20%的機率是-1 => 20% noise
    noise=np.random.uniform(size=data_size)-noise_rate
    y=np.ones(x.shape)
    for i in range(len(x)):
        y[i]=sign(x[i])*sign(noise[i])
    return y

In [4]:
def genTheta(x):
    n=len(x)
    sort=sorted(x)
    theta=np.zeros(n+1)
    theta[0]=-1   #theta切再最左邊 => 值=-1
    theta[n]=1  #theta切再最左邊 => 值=1
    for i in range(1,n-1):
        theta[i]=(sort[i]+sort[i+1])/2
    return theta

In [5]:
def calError(y, x, theta, s):
    error=0
    for i in range(len(y)):
        if y[i]!= s*sign(x[i]-theta):
            error+=1
    error=error/len(y)
    return error

### Q.17 & Q18

In [6]:
data_size=20
run=5000
noise_rate=0.2
E_in=0
E_out=0

for i in range(run):
    x=np.random.uniform(-1,1,data_size)   #x-> n dimensions
    y=genNoise(noise_rate, data_size, x)  #y-> n dimensions
    theta=genTheta(x)  #theta => n+1  dimensions
    e_in=np.zeros((2,len(theta)))  #e_in => 2*n+1, store the e_in errors for different s and theta
    s_min=0
    theta_min=0
    e_in_min=0
    for t in range(len(theta)):
        e_in[0][t]=calError(y,x,theta[t],s=1)
        e_in[1][t]=calError(y,x,theta[t],s=-1)
    if min(e_in[0])<min(e_in[1]):  #if minimum error exists when s=1
        s_min=1
        theta_min=theta[np.argmin(e_in[0])]
        e_in_min=min(e_in[0])
    else:
        s_min=-1
        theta_min=theta[np.argmin(e_in[1])]
        e_in_min=min(e_in[1])
    e_out=0.5+0.3*s_min*(np.abs(theta_min)-1)
    E_in+=e_in_min
    E_out+=e_out
    
ave_in=E_in/run
ave_out=E_out/run
print('E_in: ', ave_in, 'E_out:', ave_out)

E_in:  0.1704800000000002 E_out: 0.26064366091607255


In [7]:
#load data from file, and extract x,y
def load_data(file_path):
    f=open(file_path,'r')
    lines=f.readlines()
    sample=len(lines)  #總共的sample個數

    x = np.zeros((sample, 9))
    y = np.zeros((sample))
    index=0

    for line in lines:
        item=line.strip().split()
        x[index]=np.array([float(i) for i in item[0:9]])
        y[index]=float(item[9])
        index+=1
    
    return x,y

In [8]:
#mostly copy from Q17 & Q18, given x from certain dimension, find the s, theta, and E_in
def deciStump(x,y):
    theta=genTheta(x)
    e_in=np.zeros((2,len(theta)))  #e_in => 2*n+1, store the e_in errors for different s and theta
    s_min=0
    theta_min=0
    e_in_min=0
    for t in range(len(theta)):
        e_in[0][t]=calError(y,x,theta[t],s=1)
        e_in[1][t]=calError(y,x,theta[t],s=-1)
    if min(e_in[0])<min(e_in[1]):  #if minimum error exists when s=1
        s_min=1
        theta_min=theta[np.argmin(e_in[0])]
        e_in_min=min(e_in[0])
    else:
        s_min=-1
        theta_min=theta[np.argmin(e_in[1])]
        e_in_min=min(e_in[1])
    return s_min, theta_min, e_in_min

In [9]:
#extract data x from certain dimension (dim)
#convert x from (100,9) to (100)
def x_dim(x, dim):
    return np.reshape(x[:,dim], len(x))

### Q.19 & Q.20

In [10]:
x_train, y_train = load_data('hw2_train.dat.txt')
#find the best s, theta, E_in from each dimension
s, s_min, theta, theta_min, E_in, E_in_min, dimension=0,0,0,0,0,1,0
for i in range(9):
    x=x_dim(x_train,i)
    s,theta,E_in=deciStump(x,y_train)
    if E_in<E_in_min:
        E_in_min=E_in
        s_min=s
        theta_min=theta
        dimension=i
print('E_in_min: ', E_in_min)

x_test, y_test = load_data('hw2_test.dat.txt')
x_test=x_dim(x_test, dimension)
E_out=calError(y_test, x_test, theta_min, s_min)
print('E_out: ', E_out)

E_in_min:  0.25
E_out:  0.355
